### Introduction
Data cleaning and feature engineering is an essential part in working with data. Recently, I have been reviewing some past personal projects and refactoring them. One of these projects is the [visualization](https://towardsdatascience.com/data-visualization-using-matplotlib-seaborn-97f788f18084?source=friends_link&sk=d73c37a5b892d447d40d0da349ce4ec0) of iaito order details.

In this article, using an updated version of the dataset, we'll go through several examples of data  operations, what happens to the dataframe with the operations and considerations that may not be that apparent. An overview of the operations to be discussed are:

* apply
* map
* applymap
* for loops
* iterrows
* itertuples
* vectorization

In [29]:
# import libraries
import pandas as pd
import numpy as np
import re
pd.set_option('display.max_columns', None)

### Data set
The dataset details the records of owners and the technical specifications of their iaito. Since the last project, several new entries have been added. 

In [2]:
# load the dataset
df = pd.read_excel('project_nanato2022_Part1.xlsx')
print(f'df rows: {df.shape[0]}, df columns: {df.shape[1]}')
df.head(1)

df rows: 47, df columns: 39


,Date,Owner,Gender,Height,Total,Postage,Courier,Model,Sword total price,Model Price,"Length, blade\n(shaku-sun-bu)",weight (g),Blade length (addn charge),Toushin,Hi,Hi Addn charge,Hamon,Hamon addn charge,Habaki,Tsuka ito,Ito color,"Length, Tsuka\n(Sun-bu)",Ito wrap,Same,Same addn charge,Menuki,Fuchi/\nKashira,Silver Koshirae(Addn charge),Kojiri,"Addn charge (Kojiri, Tsuba, etc.)",Tsuba,Tsuba weight(g),Saya,Saya charge,Sageo,Sageo silk charge,Bag charge,Koiguchi kun,Unnamed: 38
0,2013-11-01,Sheng Jun,M,171,72375,6075,FedEx,Chuden-S,66300,59000,2-4-5,880-940,0,(Atukuchi usuba) Thick,Yes,0,HM-1 Gunome (Single),0,Brass,Silk,TS-03,7-7,Hineri-Maki,Black,2100,MY-23 Phoenix,KY-51 Moon and ocean,0,No,0,T-44 Crescent musashi,90,SY-03 Brown matt,0,Silk,5200,0,0,NaN


In [3]:
# Check for null or missing values
df.isnull().sum()[df.isnull().sum()>0]

Unnamed: 38    44
dtype: int64

In [4]:
# Summary stats
df.describe(include='all', datetime_is_numeric=True)

,Date,Owner,Gender,Height,Total,Postage,Courier,Model,Sword total price,Model Price,"Length, blade\n(shaku-sun-bu)",weight (g),Blade length (addn charge),Toushin,Hi,Hi Addn charge,Hamon,Hamon addn charge,Habaki,Tsuka ito,Ito color,"Length, Tsuka\n(Sun-bu)",Ito wrap,Same,Same addn charge,Menuki,Fuchi/\nKashira,Silver Koshirae(Addn charge),Kojiri,"Addn charge (Kojiri, Tsuba, etc.)",Tsuba,Tsuba weight(g),Saya,Saya charge,Sageo,Sageo silk charge,Bag charge,Koiguchi kun,Unnamed: 38
count,47,47,47,47.000000,47.000000,47.000000,47,47,47.000000,47.000000,47,47,47.000000,47,47,47.000000,47,47.000000,47,47,47,47,47,47,47.000000,47,47,47.000000,47,47.000000,47,47.000000,47,47.000000,47,47.000000,47.0,47.000000,3.0
unique,NaN,42,2,NaN,NaN,NaN,2,5,NaN,NaN,8,16,NaN,2,2,NaN,19,NaN,7,6,21,7,6,4,NaN,38,34,NaN,2,NaN,33,NaN,19,NaN,2,NaN,6.0,NaN,2.0
top,NaN,Noi,M,NaN,NaN,NaN,FedEx,Chuden-S,NaN,NaN,2-4-5,880-940,NaN,(Atukuchi usuba) Thick,Yes,NaN,HM-3 Notare (Single),NaN,Brass,Silk,TS-01,7-5,Hineri-Maki,Black,NaN,MY-23 Phoenix,KY-46 Musashi,NaN,No,NaN,T-44 Crescent musashi,NaN,SY-02 Black matt,NaN,Silk,NaN,4300.0,NaN,0.0
freq,NaN,3,35,NaN,NaN,NaN,36,24,NaN,NaN,12,11,NaN,35,44,NaN,8,NaN,30,37,10,21,34,26,NaN,3,4,NaN,26,NaN,4,NaN,10,NaN,29,NaN,14.0,NaN,2.0
mean,2018-10-18 02:33:11.489361664,NaN,NaN,169.063830,106716.106383,6567.595745,NaN,NaN,100148.510638,76444.255319,NaN,NaN,417.446809,NaN,NaN,1153.191489,NaN,460.425532,NaN,NaN,NaN,NaN,NaN,NaN,1663.829787,NaN,NaN,1119.148936,NaN,4788.510638,NaN,95.638298,NaN,7689.361702,NaN,3342.553191,NaN,625.531915,NaN
min,2013-11-01 00:00:00,NaN,NaN,153.000000,47060.000000,2300.000000,NaN,NaN,40260.000000,37400.000000,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,-12000.000000,NaN,0.000000,NaN,80.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN
25%,2017-06-01 00:00:00,NaN,NaN,163.500000,79100.000000,5200.000000,NaN,NaN,73230.000000,63720.000000,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,0.000000,NaN,85.000000,NaN,0.000000,NaN,0.000000,NaN,700.000000,NaN
50%,2018-09-01 00:00:00,NaN,NaN,170.000000,96800.000000,6200.000000,NaN,NaN,90900.000000,63720.000000,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,2160.000000,NaN,NaN,0.000000,NaN,0.000000,NaN,90.000000,NaN,0.000000,NaN,5400.000000,NaN,700.000000,NaN
75%,2020-06-29 00:00:00,NaN,NaN,177.500000,132230.000000,6800.000000,NaN,NaN,125430.000000,79200.000000,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,2180.000000,NaN,NaN,0.000000,NaN,6480.000000,NaN,100.000000,NaN,10850.000000,NaN,5400.000000,NaN,700.000000,NaN
max,2021-10-01 00:00:00,NaN,NaN,182.000000,208000.000000,22856.000000,NaN,NaN,205700.000000,137500.000000,NaN,NaN,3300.000000,NaN,NaN,5500.000000,NaN,2200.000000,NaN,NaN,NaN,NaN,NaN,NaN,5500.000000,NaN,NaN,48400.000000,NaN,45100.000000,NaN,150.000000,NaN,50600.000000,NaN,5500.000000,NaN,1400.000000,NaN


It appears the `Unnamed Column` is a redundant column and can be safely dropped.
Dropping or imputing null values early on (if one can) helps in terms of streamlining the datacleaning process & in our case, reduces the memory usage as illustrated below. 

In [5]:
# note memory usage before dropping the Unnamed column
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47 entries, 0 to 46
Data columns (total 39 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   Date                               47 non-null     datetime64[ns]
 1   Owner                              47 non-null     object        
 2   Gender                             47 non-null     object        
 3   Height                             47 non-null     int64         
 4   Total                              47 non-null     int64         
 5   Postage                            47 non-null     int64         
 6   Courier                            47 non-null     object        
 7   Model                              47 non-null     object        
 8   Sword total price                  47 non-null     int64         
 9   Model Price                        47 non-null     int64         
 10  Length, blade
(shaku-sun-bu)       47 no

In [6]:
# Drop the Unnamed: 38 column
df.drop('Unnamed: 38', axis=1, inplace=True)
# Note the reduction in memory usage
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47 entries, 0 to 46
Data columns (total 38 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   Date                               47 non-null     datetime64[ns]
 1   Owner                              47 non-null     object        
 2   Gender                             47 non-null     object        
 3   Height                             47 non-null     int64         
 4   Total                              47 non-null     int64         
 5   Postage                            47 non-null     int64         
 6   Courier                            47 non-null     object        
 7   Model                              47 non-null     object        
 8   Sword total price                  47 non-null     int64         
 9   Model Price                        47 non-null     int64         
 10  Length, blade
(shaku-sun-bu)       47 no

In [7]:
# Update the column names for easier management
# review the column names
df.columns

Index(['Date', 'Owner', 'Gender', 'Height', 'Total', 'Postage', 'Courier',
       'Model', 'Sword total price', 'Model Price',
       'Length, blade\n(shaku-sun-bu)', 'weight (g)',
       'Blade length (addn charge)', 'Toushin', 'Hi ', 'Hi Addn charge',
       'Hamon', 'Hamon addn charge', 'Habaki', 'Tsuka ito', 'Ito color',
       'Length, Tsuka\n(Sun-bu)', 'Ito wrap', 'Same', 'Same addn charge',
       'Menuki', 'Fuchi/\nKashira', 'Silver Koshirae(Addn charge)', 'Kojiri',
       'Addn charge (Kojiri, Tsuba, etc.)', 'Tsuba', 'Tsuba weight(g)', 'Saya',
       'Saya charge', 'Sageo', 'Sageo silk charge', 'Bag charge',
       'Koiguchi kun'],
      dtype='object')

In [8]:
# lowercase, drop commas, replace newline & spaces with underscores
col_names = []
for old in df.columns:
    new = re.sub( r"[,]" ,'',old.strip())
    new = re.sub( r"[\n\s/]" ,'_',new)
    col_names.append(new.lower())
df.rename(columns=dict(zip(df.columns, col_names)), inplace=True)

In [9]:
df.head()

,date,owner,gender,height,total,postage,courier,model,sword_total_price,model_price,length_blade_(shaku-sun-bu),weight_(g),blade_length_(addn_charge),toushin,hi,hi_addn_charge,hamon,hamon_addn_charge,habaki,tsuka_ito,ito_color,length_tsuka_(sun-bu),ito_wrap,same,same_addn_charge,menuki,fuchi__kashira,silver_koshirae(addn_charge),kojiri,addn_charge_(kojiri_tsuba_etc.),tsuba,tsuba_weight(g),saya,saya_charge,sageo,sageo_silk_charge,bag_charge,koiguchi_kun
0,2013-11-01,Sheng Jun,M,171,72375,6075,FedEx,Chuden-S,66300,59000,2-4-5,880-940,0,(Atukuchi usuba) Thick,Yes,0,HM-1 Gunome (Single),0,Brass,Silk,TS-03,7-7,Hineri-Maki,Black,2100,MY-23 Phoenix,KY-51 Moon and ocean,0,No,0,T-44 Crescent musashi,90,SY-03 Brown matt,0,Silk,5200,0,0
1,2014-11-01,Yige,F,160,100040,6100,FedEx,Chuden-S,93940,63720,2-2-5,720-740,0,(Usukuchi) Thin,Yes,0,HM-F Sukehiro (Single),2160,Brass,Silk,TS-15,7-5,Hineri-Maki,Black,2160,MY-40 Tiger Forest,KY-37 Tiger forest,0,No,0,T-119 Toru taketora,90,SY-16 Brown tataki,16200,Silk,5400,4300,0
2,2015-03-01,Noi,F,153,79520,6100,FedEx,Chuden-S,73420,63720,2-3-0,720-740,0,(Usukuchi) Thin,Yes,0,HM-3 Notare (Single),0,Brass,Silk,TS-03,7-5,Hineri-Maki,Black,0,MY-38 Plum,KY-27 Plum,0,No,0,T-95 Miyomoto musashi,80,SY-03 Brown matt,0,Silk,5400,4300,0
3,2015-12-01,Rei,M,180,82080,5800,FedEx,Chuden-S,76280,63720,2-4-5,880-940,0,(Atukuchi usuba) Thick,Yes,0,HM-02 Suguha (Double),0,Brass,Silk,TS-01,7-8,Hineri-Maki,Black,2160,MY-46 Phoenix branch,KY-46 Musashi,0,No,0,T-140 Phoenix,115,SY-02 Black matt,0,Silk,5400,4300,700
4,2016-10-01,Warren,M,180,100540,5900,FedEx,Chuden-S,94640,63720,2-4-5,880-940,0,(Atukuchi usuba) Thick,Yes,0,HM-C Masamune (Double),2160,Brass,Silk,TS-01,8-0,Hineri-Maki,Black,2160,MY-87 Sanskrit,KY-19 Sanskrit,0,No,0,T-46 Sanskrit,90,SY-15 Black matt,16200,Silk,5400,4300,700


### Use case 1 - Owner name length
We'll go through `apply`, `map`, `applymap` and `for` loops. Among these three operations, `mapapply` may not be as commonly used.  We'll use a simple use case - counting the character length of owner name. Essentially, this is deriving a new data column from an existing one. In reality, this could be deriving advertisement length, etc. 

We'll also time the implementation and discuss these later. I used `timeit` and kept the number of loops to 100.

We'll go through the documentation for a better idea. From the documentation of `apply`, it states the application of a function along an axis of the DataFrame with Series objects passed to the function. `map` substitutes the Series value with another value. `mapapply` applies a function to the elements of the DataFrame - input need to be a dataframe; something to be keep in mind as we go through the examples.


In [10]:
# apply
%timeit -n100 df['name_L1'] = df['owner'].apply(lambda x: len(x))

269 µs ± 47.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
# map
%timeit -n100 df['name_L2'] = df['owner'].map(lambda x: len(x))

184 µs ± 27.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [12]:
# applymap - applies to elementwise, so pass in a dataframe
%timeit -n100 df['name_L3'] = df[['owner']].applymap(lambda x: len(x))

896 µs ± 65 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [13]:
%%timeit -n100
# for loop
t_name = []
for ele in df['owner']:
    t_name.append(len(ele))
df['name_L4'] = t_name

142 µs ± 39 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Depending on the use case, the optimal operations will vary. Our current use case is generating a column using an existing data column. For our use case, either `for` loops and `map` would suffice, though `for` loop implementation is more verbose. We can get away with `apply` as well. However, if we are dealing with large datasets (e.g. millions of records, `apply` is not optimal).

In [14]:
# Drop redundant columns 
df = df.iloc[:,:-3].copy()

### Use case 2 - Splitting the weight into the two ranges
The weight column consists of a range of the expected weight of the iaito. We extract the values into two new separate columns.

We'll go through two approaches:
* `for` loop and string splitting
* `.apply` method

In [15]:
%%timeit -n100
# for loops and splitting of strings
w_lower1, w_upper1 = [], []
for weight in df['weight_(g)']:
    w_lower1.append(weight.split('-')[0])
    w_upper1.append(weight.split('-')[1])
df['w_lower1(g)'] = w_lower1
df['w_upper1(g)'] = w_upper1

221 µs ± 42.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [16]:
# helper function to split weight column
def splitter(row):
    return [row.str.split('-')[0][0],  row.str.split('-')[0][1],]

In [17]:
%%timeit -n100
# .apply result_type='expand' creates a dataframe
frame = df[['weight_(g)']].apply(splitter, axis=1, result_type='expand')
frame.rename(columns={0:'w_lower2(g)',
                      1:'w_upper2(g)',
                     },inplace=True)
df[frame.columns] = frame

15.2 ms ± 408 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


To use `apply` result_type, the implementation `axis` has to be 1. In the example shown, a new dataframe is returned with 'expand'. 

In [18]:
# Drop redundant columns 
df = df.iloc[:,:-2].copy()

### Use case 3 - Deriving new data column from multiple columns
For this example, we'll create a new name column with the title followed by the owner name. The title we could infer from the gender column.
We'll go through these approaches:
* `for` loop
* `iterrows`
* `itertuples`
* `list comprehension` + `apply`
* vector instructions


In [134]:
%%timeit -n100
# for loop
res = []
for i in range(len(df['gender'])):    
    if df['gender'][i] == 'M':
        res.append('Mr. ' + df['owner'][i])
    else:
        res.append('Ms. ' + df['owner'][i])
df['name1'] = res

661 µs ± 144 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


`iterrows` as its name implies, iterates through dataframe rows. It does so as a index, series pair. Let's take a look. 

In [22]:
# cap at the 1st data record
for ele in df[['owner', 'gender']].iloc[:1].iterrows():
    print(ele,'\n')
    print(f"index\n{ele[0]}",'\n')
    print(f"series\n{ele[1]}",'\n')

(0, owner     Sheng Jun
gender            M
Name: 0, dtype: object) 

index
0 

series
owner     Sheng Jun
gender            M
Name: 0, dtype: object 



In [44]:
%%timeit -n100
# iterrows implementation
res = []
for ele in df[['owner', 'gender']].iterrows():
    if ele[1][1] == 'M':
        res.append('Mr. ' + ele[1][0])
    else:
        res.append('Ms. ' + ele[1][0])
df['name2'] = res

3.55 ms ± 505 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


`itertuples` iterates over the dataframe as named tuples. Turning the default `index` off shifts the first column values into the index. It's faster than `iterrows`. Let's take a look.

In [24]:
# cap at 1st record, default index on
for ele in df[['owner', 'gender']].iloc[:1].itertuples():
    print(f"index\n{ele[0]}")
    print(f"tuples\n{ele[1], ele[2]}")

index
0
tuples
('Sheng Jun', 'M')


In [25]:
# cap at first record, index off
for ele in df[['owner', 'gender']].iloc[:1].itertuples(index=False):
    print(f"index\n{ele[0]}")
    print(f"tuples\n{ele[1]}")

index
Sheng Jun
tuples
M


In [46]:
%%timeit -n100
res = []
# itertuples implementation
for ele in df[['owner', 'gender']].itertuples(index=False):
    if ele[1] == 'M':
        res.append('Mr. ' + ele[0])
    else:
        res.append('Ms. ' + ele[0])
df['name3'] = res

931 µs ± 157 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [47]:
%%timeit -n100
# combination of list comprehension and apply
title_ = ['Mr. ' if x == 'M' else 'Ms. ' for x in df['gender'] ]
df['t'] = title_
def name_(row):
    return row['t'] + row['owner']
df['name4'] = df.apply(name_,axis=1)

1.77 ms ± 154 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Last in our example, vectorization would help minimize running time of code like so:

In [135]:
%%timeit -n100
# vectorization
arr1 = df['owner'].array
arr2 = df['gender'].array
arr3 = []
for i in range(len(arr1)):
    if arr2[i] == 'M':
        arr3.append('Mr. ' + arr1[i])
    else:
        arr3.append('Ms. ' + arr1[i])
df['name5'] = arr3

281 µs ± 62.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


More project spin-offs from this dataset in the pipeline, but that's it for now. Thanks for reading.